<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/contratacao_banc_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predição da probabilidade de um cliente contratar um produto bancario


Etapas:
1.   Data prep
2.   Feature Selection
3.   Avaliação do SVM


In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

### Data understanding

In [2]:
# Data understanding
DATA = pd.read_csv('/content/bank-additional-full.csv',sep=';')
DATA.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
DATA.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [4]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,age,int64,0,0.0,78
1,previous,int64,0,0.0,8
2,pdays,int64,0,0.0,27
3,campaign,int64,0,0.0,42
4,duration,int64,0,0.0,1544
5,nr.employed,float64,0,0.0,11
6,emp.var.rate,float64,0,0.0,10
7,cons.price.idx,float64,0,0.0,26
8,cons.conf.idx,float64,0,0.0,26
9,euribor3m,float64,0,0.0,316


In [37]:
# Realizando HOLD-OUT da base, buscando conseguir avaliar o modelo posteriormente.
lista_spec = ['y']
target = DATA['y']
data_var = DATA.drop(lista_spec,axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(data_var,
                                                    target,
                                                    test_size=0.3,
                                                    random_state=40)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape


((28831, 20), (12357, 20), (28831,), (12357,))

### Tratando o TARGET do treino com label encolding e aplicando no teste

In [38]:
card_cutoff = 7
df_categ_labelenc = metadados[(metadados['cardinalidade'] > card_cutoff) & (metadados['tipo'] == 'object')]
lista_vars_abt = list(X_train.columns)
lista_lenc = list(df_categ_labelenc.nome_variavel.values)


print('Lista de vars para Label Encoding: ',lista_lenc)

Lista de vars para Label Encoding:  ['education', 'job', 'month']


In [45]:
from sklearn.preprocessing import LabelEncoder

encoders = {}

for col in lista_lenc:
    encoder = LabelEncoder()
    X_train[col] = encoder.fit_transform(X_train[col])

    # Armazena o encoder para a coluna atual em um dicionário
    encoders[col] = encoder

# Salva o dicionário de encoders e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoders': encoders,
    'columns': lista_lenc
}

with open('/content/labelenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)
data_to_serialize

{'encoders': {'education': LabelEncoder(),
  'job': LabelEncoder(),
  'month': LabelEncoder()},
 'columns': ['education', 'job', 'month']}

In [42]:
X_train.head(), X_train.shape

(       age            job  marital            education  default housing loan  \
 10944   29  self-employed  married  professional.course       no      no   no   
 21196   37     technician  married    university.degree       no      no   no   
 24943   36     technician  married          high.school       no      no   no   
 420     57    blue-collar  married             basic.4y  unknown     yes   no   
 34036   33         admin.   single    university.degree       no     yes   no   
 
          contact month day_of_week  duration  campaign  pdays  previous  \
 10944  telephone   jun         wed       632         1    999         0   
 21196   cellular   aug         mon       124         2    999         0   
 24943   cellular   nov         tue        91         2    999         1   
 420    telephone   may         tue       208         1    999         0   
 34036   cellular   may         wed       484         3    999         0   
 
           poutcome  emp.var.rate  cons.price.id

In [39]:
# Carregar os encoders e a lista de colunas
with open('/content/labelenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# APLICANDO O LABELENCOLDING NA BASE DE TESTE
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        X_test[col] = loaded_encoders[col].transform(X_test[col])



(       age  job  marital  education  default housing loan    contact  month  \
 39150   35    7   single          3       no     yes   no  telephone      5   
 26023   43    4  married          6       no      no  yes   cellular      7   
 26948   31    1  married          2       no      no   no   cellular      7   
 29746   44    4  married          6       no      no   no   cellular      0   
 2451    38    1  married          0  unknown     yes   no  telephone      6   
 
       day_of_week  duration  campaign  pdays  previous     poutcome  \
 39150         tue       187         3    999         0  nonexistent   
 26023         wed        73         2    999         1      failure   
 26948         thu       693         3    999         0  nonexistent   
 29746         mon        44         2    999         0  nonexistent   
 2451          tue       173         1    999         0  nonexistent   
 
        emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed  
 39150

In [41]:
metadados = generate_metadata(X_train)


from sklearn.preprocessing import OneHotEncoder

card_cutoff = 7
df_categ_onehot = metadados[(metadados['cardinalidade'] <= card_cutoff) & (metadados['tipo'] == 'object')]
lista_onehot = list(df_categ_onehot.nome_variavel.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Instanciando o encoder
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplicando o one-hot encoding
encoded_data = encoder.fit_transform(X_train[lista_onehot])
encoded_cols = encoder.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=X_train.index)

X_train_lab_one = pd.concat([X_train.drop(lista_onehot, axis=1), encoded_df], axis=1)

# Salva o encoder e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoder': encoder,
    'columns': lista_onehot
}

with open('/content/onehotenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)
X_train_lab_one.head()

Lista de vars para OneHot Encoding:  ['contact', 'loan', 'housing', 'default', 'poutcome', 'marital', 'day_of_week']


,age,job,education,month,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,...,default_yes,poutcome_nonexistent,poutcome_success,marital_married,marital_single,marital_unknown,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
10944,29,self-employed,professional.course,jun,632,1,999,0,1.4,94.465,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
21196,37,technician,university.degree,aug,124,2,999,0,1.4,93.444,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
24943,36,technician,high.school,nov,91,2,999,1,-0.1,93.200,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
420,57,blue-collar,basic.4y,may,208,1,999,0,1.1,93.994,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
34036,33,admin.,university.degree,may,484,3,999,0,-1.8,92.893,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [51]:
X_train_lab_one = pd.concat([X_train.drop(lista_onehot, axis=1), encoded_df], axis=1)


Index(['age', 'job', 'education', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'contact_telephone', 'loan_unknown',
       'loan_yes', 'housing_unknown', 'housing_yes', 'default_unknown',
       'default_yes', 'poutcome_nonexistent', 'poutcome_success',
       'marital_married', 'marital_single', 'marital_unknown',
       'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed'],
      dtype='object')

In [46]:
# Carregar o encoder e a lista de colunas
with open('/content/onehotenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Suponha test_df como sua base de teste
encoded_data_test = loaded_encoder.transform(X_test[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=X_test.index)

X_test_lab_one = pd.concat([X_test.drop(loaded_columns, axis=1), encoded_df_test], axis=1)
X_test_lab_one.head()

,age,job,education,month,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,...,default_yes,poutcome_nonexistent,poutcome_success,marital_married,marital_single,marital_unknown,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
39150,35,7,3,5,187,3,999,0,-1.8,93.369,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
26023,43,4,6,7,73,2,999,1,-0.1,93.200,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
26948,31,1,2,7,693,3,999,0,-0.1,93.200,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
29746,44,4,6,0,44,2,999,0,-1.8,93.075,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2451,38,1,0,6,173,1,999,0,1.1,93.994,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [52]:
X_test_lab_one.shape

(12357, 29)

In [59]:
#Normalização

from sklearn.preprocessing import MinMaxScaler


from sklearn.preprocessing import StandardScaler
# Instanciando o scaler
scaler = MinMaxScaler()

# Padronizando a base de treino
X_train_scaled = scaler.fit_transform(X_train_lab_one)
X_train_lab_one_norm = pd.DataFrame(X_train_scaled, columns=X_train_lab_one.columns, index=X_train_lab_one.index)

# Salva o scaler em um arquivo .pkl
with open('/content/scaler_norm.pkl', 'wb') as f:
    pickle.dump(scaler, f)
X_train_lab_one_norm.head()

,age,job,education,month,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,...,default_yes,poutcome_nonexistent,poutcome_success,marital_married,marital_single,marital_unknown,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
10944,0.148148,0.545455,0.714286,0.444444,0.128508,0.000000,1.0,0.000000,1.000000,0.882307,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
21196,0.246914,0.818182,0.857143,0.111111,0.025214,0.023810,1.0,0.000000,1.000000,0.484412,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
24943,0.234568,0.818182,0.428571,0.777778,0.018503,0.023810,1.0,0.142857,0.687500,0.389322,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
420,0.493827,0.090909,0.000000,0.666667,0.042294,0.000000,1.0,0.000000,0.937500,0.698753,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
34036,0.197531,0.000000,0.857143,0.666667,0.098414,0.047619,1.0,0.000000,0.333333,0.269680,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [60]:
# Carregar o scaler
with open('/content/scaler_norm.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Suponha test_df como sua base de teste
X_test_scaled = loaded_scaler.transform(X_test_lab_one)
X_test_lab_one_norm = pd.DataFrame(X_test_scaled, columns=X_test_lab_one.columns, index=X_test_lab_one.index)
X_test_lab_one_norm.head()

,age,job,education,month,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,...,default_yes,poutcome_nonexistent,poutcome_success,marital_married,marital_single,marital_unknown,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
39150,0.222222,0.636364,0.428571,0.555556,0.038024,0.047619,1.0,0.000000,0.333333,0.455183,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
26023,0.320988,0.363636,0.857143,0.777778,0.014843,0.023810,1.0,0.142857,0.687500,0.389322,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
26948,0.172840,0.090909,0.285714,0.777778,0.140911,0.047619,1.0,0.000000,0.687500,0.389322,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
29746,0.333333,0.363636,0.857143,0.000000,0.008947,0.023810,1.0,0.000000,0.333333,0.340608,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2451,0.259259,0.090909,0.000000,0.666667,0.035177,0.000000,1.0,0.000000,0.937500,0.698753,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


### feature selection

In [65]:
# FEATURE SELECTION
from sklearn.ensemble import RandomForestClassifier

X_f = X_train_lab_one_norm
Y_f = Y_train

clf = RandomForestClassifier() # troque aqui qual classificador quer utilizar: GradientBoostingClassifier, DecisionTreeClassifier
clf.fit(X_f, Y_f)

# Obter importância das variáveis
feature_importances = clf.feature_importances_
features = pd.DataFrame({
    'Feature': X_f.columns,
    'Importance': feature_importances
})

# Ordenar variáveis por importância
features = features.sort_values(by='Importance', ascending=False)

features

,Feature,Importance
4,duration,3.066469e-01
11,euribor3m,1.042642e-01
0,age,9.149277e-02
12,nr.employed,6.163088e-02
1,job,4.881305e-02
2,education,4.418005e-02
5,campaign,4.235919e-02
10,cons.conf.idx,3.331788e-02
6,pdays,3.104953e-02
21,poutcome_success,2.656639e-02


from matplotlib import pyplot as plt
features['Importance'].plot(kind='hist', bins=20, title='Importance')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
features['Importance'].plot(kind='line', figsize=(8, 4), title='Importance')
plt.gca().spines[['top', 'right']].set_visible(False)

In [66]:
# Estabelecer um limite de corte, por exemplo, x% da importância máxima
cutoff = 0.01
#cutoff = cutoff_maximp * feature_importances.max()
# Selecionar variáveis acima do corte
selected_features = features['Feature'][features['Importance'] > cutoff].tolist()

X_train_lab_one_norm_feat = X_train_lab_one_norm.loc[:,selected_features]

X_train_lab_one_norm_feat.shape

(28831, 22)

In [67]:
# Salvar a lista em um arquivo .pkl
import pickle
with open('/content/features_selec.pkl', 'wb') as f:
    pickle.dump(selected_features, f)

In [68]:
# Carregar a lista do arquivo .pkl
with open('/content/features_selec.pkl', 'rb') as f:
    loaded_lista = pickle.load(f)

X_test_lab_one_norm_feat = X_test_lab_one_norm[loaded_lista]

X_test_lab_one_norm_feat.head()

,duration,euribor3m,age,nr.employed,job,education,campaign,cons.conf.idx,pdays,poutcome_success,...,month,emp.var.rate,marital_married,loan_yes,previous,day_of_week_thu,day_of_week_tue,day_of_week_mon,marital_single,day_of_week_wed
39150,0.038024,0.004761,0.222222,0.170510,0.636364,0.428571,0.047619,0.669456,1.0,0.0,...,0.555556,0.333333,0.0,0.0,0.000000,0.0,1.0,0.0,1.0,0.0
26023,0.014843,0.790297,0.320988,0.877883,0.363636,0.857143,0.023810,0.368201,1.0,0.0,...,0.777778,0.687500,1.0,1.0,0.142857,0.0,0.0,0.0,0.0,1.0
26948,0.140911,0.780322,0.172840,0.877883,0.090909,0.285714,0.047619,0.368201,1.0,0.0,...,0.777778,0.687500,1.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0
29746,0.008947,0.174790,0.333333,0.512287,0.363636,0.857143,0.023810,0.154812,1.0,0.0,...,0.000000,0.333333,1.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0
2451,0.035177,0.957153,0.259259,0.859735,0.090909,0.000000,0.000000,0.602510,1.0,0.0,...,0.666667,0.937500,1.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0


In [69]:
Y_train.value_counts()

y
no     25645
yes     3186
Name: count, dtype: int64

In [70]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

x_treino_final = X_train_lab_one_norm_feat
y_treino_final = Y_train
x_teste_final = X_test_lab_one_norm_feat
y_teste_final = Y_test

# Definindo o modelo
model = SVC(probability=True)

# Definindo os parâmetros para o grid search
param_grid = {
    'C': [0.1, 1],
    'kernel': ['linear'],
    # 'degree': [2, 3],  # Só é relevante se kernel='poly'
    'gamma': [0.1, 1]
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=5, # diminui o numero de folds por conta do baixo número de observações
                    verbose=2,
                    scoring='roc_auc',  # 'accuracy' 'precision' 'recall' 'roc_auc' 'f1'
                    n_jobs=-1)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor AUC: ", grid.best_score_)

# Utilizando o melhor modelo para fazer previsões
predictions = grid.best_estimator_.predict(x_teste_final)



best_model = grid.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores Parâmetros:  {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Melhor AUC:  0.9275728274848509


In [72]:
with open('/content/best_model.pkl', 'wb') as file:
  pickle.dump(best_model, file)
best_model

SVC(C=1, gamma=0.1, kernel='linear', probability=True)

In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report


predict = best_model.predict(x_teste_final)

print(confusion_matrix(y_teste_final, predict))
print(accuracy_score(y_teste_final, predict))
print(classification_report(y_teste_final, predict))

[[10737   166]
 [ 1126   328]]
0.8954438779639071
              precision    recall  f1-score   support

          no       0.91      0.98      0.94     10903
         yes       0.66      0.23      0.34      1454

    accuracy                           0.90     12357
   macro avg       0.78      0.61      0.64     12357
weighted avg       0.88      0.90      0.87     12357



In [80]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, auc

def calculate_metrics(nm_modelo, model, X_train, y_train, X_test, y_test):
    # Fazendo predições
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculando as métricas para o conjunto de treino

    auc_roc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])


    gini_index_train = 2 * auc_roc_train - 1


    auc_roc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])


    gini_index_test = 2 * auc_roc_test - 1

    # Criando o DataFrame com as métricas calculadas
    metrics_df = pd.DataFrame({

        'GINI': [gini_index_train, gini_index_test],

    })

    return metrics_df


# Uso da função
metrics_df = calculate_metrics('SVM',best_model, x_treino_final, y_treino_final, x_teste_final , y_teste_final )
metrics_df



,GINI
0,0.856736
1,0.854971


### Conclusão: Apesar do GINI ser alto, acredito que poderíamos melhorar esse modelo focando na predição do evento (target = 1/'yes'), conforme esperado, o SVM demorou 17 minutos para gerar poucos modelos para teste, não sendo um bom modelo para ser implementado em uma rotina que precise de alta tempestividade